In [3]:
#Openmanipulator API

import rclpy
from rclpy.node import Node

from geometry_msgs.msg import PoseStamped
from trajectory_msgs.msg import JointTrajectory, JointTrajectoryPoint
from moveit_msgs.srv import GetPositionIK

class RobotInterface(Node):
    def __init__(self):
        super().__init__("robot_interface")

        # === MoveIt2 IK 서비스 연결 ===
        self.get_logger().info("Connecting to MoveIt2 /compute_ik service...")
        self.ik_client = self.create_client(GetPositionIK, "/compute_ik")
        while not self.ik_client.wait_for_service(timeout_sec=2.0):
            self.get_logger().info("⏳ Waiting for /compute_ik ...")

        # === JointTrajectory Publisher (Arm, Gripper) ===
        self.arm_pub = self.create_publisher(
            JointTrajectory, "/joint_trajectory", 10
        )
        self.gripper_pub = self.create_publisher(
            JointTrajectory, "/gripper_controller/joint_trajectory", 10
        )

        # === 기본 설정 ===
        self.joint_names = ["joint1", "joint2", "joint3", "joint4"]  # 실제 로봇 확인 필요
        self.get_logger().info("✅ RobotInterface initialized (Humble version)")

    # 1️⃣ 목표 pose로 이동
    def move_to_pose(self, pose: PoseStamped):
        """EE를 특정 Pose로 이동 (MoveIt의 /compute_ik 서비스 이용)"""
        self.get_logger().info("🚀 Requesting IK for target pose...")
        req = GetPositionIK.Request()
        req.ik_request.group_name = "arm"
        req.ik_request.pose_stamped = pose
        req.ik_request.avoid_collisions = False

        future = self.ik_client.call_async(req)
        rclpy.spin_until_future_complete(self, future)
        res = future.result()

        if res.error_code.val != res.error_code.SUCCESS:
            self.get_logger().warn("⚠️ No valid IK solution found!")
            return False

        joint_positions = res.solution.joint_state.position
        self.get_logger().info(f"✅ IK solved: {joint_positions}")

        traj = JointTrajectory()
        traj.joint_names = self.joint_names
        point = JointTrajectoryPoint()
        point.positions = joint_positions
        point.time_from_start.sec = 2
        traj.points.append(point)
        self.arm_pub.publish(traj)
        self.get_logger().info("🤖 Executing arm trajectory...")
        return True

    # 2️⃣ 그리퍼 open
    def open_gripper(self):
        self.get_logger().info("Opening gripper...")
        traj = JointTrajectory()
        traj.joint_names = ["gripper"]
        point = JointTrajectoryPoint()
        point.positions = [0.01]  # 열림 각도 (수정 가능)
        point.time_from_start.sec = 1
        traj.points.append(point)
        self.gripper_pub.publish(traj)
        self.get_logger().info("✅ Gripper opened")

    # 3️⃣ 그리퍼 close
    def close_gripper(self):
        self.get_logger().info("Closing gripper...")
        traj = JointTrajectory()
        traj.joint_names = ["gripper"]
        point = JointTrajectoryPoint()
        point.positions = [0.0]  # 닫힘 각도 (수정 가능)
        point.time_from_start.sec = 1
        traj.points.append(point)
        self.gripper_pub.publish(traj)
        self.get_logger().info("✅ Gripper closed")

    # 4️⃣ Home pose 이동
    def move_home(self):
        """단순히 조인트별 Home 각도로 이동"""
        self.get_logger().info("Moving to 'home' position...")
        home_joints = [0.0, -1.05, 0.35, 0.7]  # 실제 home pose 각도
        traj = JointTrajectory()
        traj.joint_names = self.joint_names
        point = JointTrajectoryPoint()
        point.positions = home_joints
        point.time_from_start.sec = 2
        traj.points.append(point)
        self.arm_pub.publish(traj)
        self.get_logger().info("✅ Home position reached")

    # 5️⃣ 미리 정의된 pose로 이동
    def move_named_pose(self, pose_name: str):
        """MoveIt SRDF에 정의된 pose가 있다면 /compute_ik로 요청 가능"""
        self.get_logger().info(f"Moving to named pose: {pose_name}")
        # 실제로는 move_home 등으로 대응하거나 별도 DB로 관리
        if pose_name == "ready":
            pose = PoseStamped()
            pose.header.frame_id = "base_link"
            pose.pose.position.x = 0.2
            pose.pose.position.y = 0.0
            pose.pose.position.z = 0.1
            pose.pose.orientation.w = 1.0
            self.move_to_pose(pose)
        else:
            self.get_logger().warn(f"⚠️ Pose '{pose_name}' not defined locally")

    # 6️⃣ 비상정지
    def emergency_stop(self):
        self.get_logger().warn("🛑 Emergency stop triggered!")
        # Humble에서는 MoveItPy.stop() 불가 → 단순히 topic publish 중단만
        self.arm_pub.destroy()
        self.get_logger().info("✅ Publishers stopped")

    # === 단독 실행 테스트용 ===
    def test_sequence(self):
        self.move_home()
        self.open_gripper()
        self.close_gripper()
        pose = PoseStamped()
        pose.header.frame_id = "base_link"
        pose.pose.position.x = 0.18
        pose.pose.position.y = 0.0
        pose.pose.position.z = 0.08
        pose.pose.orientation.w = 1.0
        self.move_to_pose(pose)


def main(args=None):
    rclpy.init(args=args)
    node = RobotInterface()
    node.test_sequence()  # 단독 테스트 실행
    node.destroy_node()
    rclpy.shutdown()


if __name__ == "__main__":
    main()

[INFO] [1761027489.982577028] [robot_interface]: Connecting to MoveIt2 /compute_ik service...
[INFO] [1761027490.063087474] [robot_interface]: ✅ RobotInterface initialized (Humble version)
[INFO] [1761027490.065234320] [robot_interface]: Moving to 'home' position...
[INFO] [1761027490.067589030] [robot_interface]: ✅ Home position reached
[INFO] [1761027490.069631891] [robot_interface]: Opening gripper...
[INFO] [1761027490.071675254] [robot_interface]: ✅ Gripper opened
[INFO] [1761027490.073659308] [robot_interface]: Closing gripper...
[INFO] [1761027490.075801788] [robot_interface]: ✅ Gripper closed
[INFO] [1761027490.077877268] [robot_interface]: 🚀 Requesting IK for target pose...
[WARN] [1761027490.081721710] [robot_interface]: ⚠️ No valid IK solution found!
